# PA2 Problem 2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import sys
sys.path.append(os.path.abspath('/content/drive/MyDrive/2_pose_estimation'))

from model import *
import torch
import numpy as np
import torchvision
import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import cv2
import math
import torch.nn as nn
import torch.optim as optim

from glob import glob
import pandas
import matplotlib.pyplot as plt

device = 'cuda:0'
num_joints = 21

In [3]:
class Dataset(Dataset):
    def __init__(self, method=None):
        self.root = '/content/drive/MyDrive/2_pose_estimation/dataset/'
        self.x_data = []
        self.y_data = []
        if method == 'train':
            self.root = self.root + 'train/'
            self.img_path = sorted(glob(self.root + 'rgb/*.jpg'))

        elif method == 'test':
            self.root = self.root + 'test/'
            self.img_path = sorted(glob(self.root + 'rgb/*.jpg'))

        for i in tqdm.tqdm(range(len(self.img_path))):
            img = cv2.imread(self.img_path[i], cv2.IMREAD_COLOR)
            b, g, r = cv2.split(img)
            img = cv2.merge([r, g, b])
            self.x_data.append(img)

            num = self.img_path[i].split('.')[0].split('/')[-1]
            img_pkl = self.root + 'meta/' + str(num) + '.pkl'
            pkl = pandas.read_pickle(img_pkl)
            coords_2d = pkl['coords_2d']
            self.y_data.append(coords_2d)

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        transform1 = torchvision.transforms.ToTensor()
        new_x_data = transform1(self.x_data[idx])

        return new_x_data, self.y_data[idx]

In [5]:
class Trainer(object):
    def __init__(self, epochs, batch_size, lr):
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = lr
        self._build_model()

        self.cost = nn.MSELoss()
        self.optimizer = optim.Adam(self.poseNet.parameters(), lr=self.learning_rate)

        dataset = Dataset(method='train')
        self.root = dataset.root
        self.dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        # Load of pretrained_weight file
        weight_PATH = '/content/drive/MyDrive/2_pose_estimation/dataset/pretrained_weight.pth'
        self.poseNet.load_state_dict(torch.load(weight_PATH))

    def _build_model(self):
        # 2d pose estimator
        poseNet = CPM2DPose()
        self.poseNet = poseNet.to(device)
        self.poseNet.train()

    def skeleton2heatmap(self, _heatmap, keypoint_targets):
        heatmap_gt = torch.zeros_like(_heatmap, device=_heatmap.device)

        keypoint_targets = (((keypoint_targets)) // 8)
        for i in range(keypoint_targets.shape[0]):
            for j in range(21):
                x = int(keypoint_targets[i, j, 0])
                y = int(keypoint_targets[i, j, 1])
                heatmap_gt[i, j, x, y] = 1

        heatmap_gt = heatmap_gt.detach().cpu().numpy()
        for i in range(keypoint_targets.shape[0]):
            for j in range(21):
                heatmap_gt[i, j, :, :] = cv2.GaussianBlur(heatmap_gt[i, j, :, :], ksize=(3, 3), sigmaX=2,
                                                          sigmaY=2) * 9 / 1.1772
        heatmap_gt = torch.FloatTensor(heatmap_gt).to(device)
        return heatmap_gt


    def heatmap2skeleton(self, heatmapsPoseNet):
        skeletons = np.zeros((heatmapsPoseNet.shape[0], heatmapsPoseNet.shape[1], 2))
        for m in range(heatmapsPoseNet.shape[0]):
            for i in range(heatmapsPoseNet.shape[1]):
                u, v = np.unravel_index(np.argmax(heatmapsPoseNet[m][i]), (32, 32))
                skeletons[m, i, 0] = u * 8
                skeletons[m, i, 1] = v * 8
        return skeletons

    # Prob 2
    def train(self):
        date = '202211'
        for epoch in tqdm.tqdm(range(self.epochs + 1)):
            totalLoss = 0
            if epoch % 2 == 0:
                torch.save(self.poseNet.state_dict(), self.root + "_".join(['model_', str(epoch), '.pth']))

            for batch_idx, samples in enumerate(self.dataloader):
                ##--------------------------
                # Implement your code here
                x_train, y_train = samples
                
                self.poseNet.eval()

                heatmapsPoseNet = self.poseNet(x_train.cuda()) # pred value
                loss = self.cost(heatmapsPoseNet, self.skeleton2heatmap(heatmapsPoseNet, y_train)) # calculate loss

                # leaning
                self.optimizer.zero_grad()
                loss.backward() 
                self.optimizer.step() 
                totalLoss += loss.item()

                print(f"Epoch {epoch}/{self.epochs}, Batch {batch_idx} loss is {loss.item()} \n")
                ##--------------------------

                ## Write train result
                if batch_idx % 20 == 0: # 21개의 keypoint 
                    with open('train_result_' + date + '.txt', 'a') as f:
                        f.write('Epoch {:4d}/{} Batch {}/{}\n'.format(
                            epoch, self.epochs, batch_idx, len(self.dataloader)
                        ))
                    print('Epoch {:4d}/{} Batch {}/{}'.format(
                        epoch, self.epochs, batch_idx, len(self.dataloader)
                    ))

            print('Loss is {}'.format(
                totalLoss/len(self.dataloader)
            ))
        
        # save trained weight
        train_path = self.root + "finetuned_{}_{}_{}.pth".format(self.epochs, self.batch_size, self.learning_rate)
        torch.save(self.poseNet.state_dict(), train_path)
        print('Finish training.')

In [6]:
epochs = 60
batchSize = 20
learningRate = 2e-5

# Prob 2
trainer = Trainer(epochs, batchSize, learningRate)
trainer.train()

  0%|          | 0/61 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').


Epoch 0/60, Batch 0 loss is 0.003910071682184935 

Epoch    0/60 Batch 0/118
Epoch 0/60, Batch 1 loss is 0.0036857572849839926 

Epoch 0/60, Batch 2 loss is 0.003935523331165314 

Epoch 0/60, Batch 3 loss is 0.004071047063916922 

Epoch 0/60, Batch 4 loss is 0.0040750461630523205 

Epoch 0/60, Batch 5 loss is 0.0038478272035717964 

Epoch 0/60, Batch 6 loss is 0.0036663925275206566 

Epoch 0/60, Batch 7 loss is 0.0036416680086404085 

Epoch 0/60, Batch 8 loss is 0.003994859755039215 

Epoch 0/60, Batch 9 loss is 0.0038480639923363924 

Epoch 0/60, Batch 10 loss is 0.0039742630906403065 

Epoch 0/60, Batch 11 loss is 0.0042217629961669445 

Epoch 0/60, Batch 12 loss is 0.0037100494373589754 

Epoch 0/60, Batch 13 loss is 0.00365069811232388 

Epoch 0/60, Batch 14 loss is 0.0037523291539400816 

Epoch 0/60, Batch 15 loss is 0.00378814316354692 

Epoch 0/60, Batch 16 loss is 0.003760470310226083 

Epoch 0/60, Batch 17 loss is 0.003793671727180481 

Epoch 0/60, Batch 18 loss is 0.004019587

  2%|▏         | 1/61 [00:15<15:29, 15.49s/it]

Epoch 0/60, Batch 116 loss is 0.0038008345291018486 

Epoch 0/60, Batch 117 loss is 0.004035244230180979 

Loss is 0.003816846730503238
Epoch 1/60, Batch 0 loss is 0.0035296669229865074 

Epoch    1/60 Batch 0/118
Epoch 1/60, Batch 1 loss is 0.0036754105240106583 

Epoch 1/60, Batch 2 loss is 0.0037449293304234743 

Epoch 1/60, Batch 3 loss is 0.0038128087762743235 

Epoch 1/60, Batch 4 loss is 0.0035602967254817486 

Epoch 1/60, Batch 5 loss is 0.003553432412445545 

Epoch 1/60, Batch 6 loss is 0.003646910423412919 

Epoch 1/60, Batch 7 loss is 0.003356287954375148 

Epoch 1/60, Batch 8 loss is 0.0038431165739893913 

Epoch 1/60, Batch 9 loss is 0.0038502321112900972 

Epoch 1/60, Batch 10 loss is 0.003766380250453949 

Epoch 1/60, Batch 11 loss is 0.003736905287951231 

Epoch 1/60, Batch 12 loss is 0.0037192159797996283 

Epoch 1/60, Batch 13 loss is 0.0036525067407637835 

Epoch 1/60, Batch 14 loss is 0.003709196811541915 

Epoch 1/60, Batch 15 loss is 0.003515455173328519 

Epoch 1

  3%|▎         | 2/61 [00:28<13:35, 13.82s/it]

...

 98%|█████████▊| 60/61 [12:44<00:12, 12.72s/it]

Epoch 59/60, Batch 115 loss is 0.0012906694319099188 

Epoch 59/60, Batch 116 loss is 0.0013340283185243607 

Epoch 59/60, Batch 117 loss is 0.0018760402454063296 

Loss is 0.0013931631051414346
Epoch 60/60, Batch 0 loss is 0.0012256662594154477 

Epoch   60/60 Batch 0/118
Epoch 60/60, Batch 1 loss is 0.0016115348553285003 

Epoch 60/60, Batch 2 loss is 0.0015222893562167883 

Epoch 60/60, Batch 3 loss is 0.0013596884673461318 

Epoch 60/60, Batch 4 loss is 0.0014109258772805333 

Epoch 60/60, Batch 5 loss is 0.00143205386120826 

Epoch 60/60, Batch 6 loss is 0.0013882750645279884 

Epoch 60/60, Batch 7 loss is 0.001428097253665328 

Epoch 60/60, Batch 8 loss is 0.0015298268990591168 

Epoch 60/60, Batch 9 loss is 0.0015240601496770978 

Epoch 60/60, Batch 10 loss is 0.001648174598813057 

Epoch 60/60, Batch 11 loss is 0.0016031055711209774 

Epoch 60/60, Batch 12 loss is 0.0014176509575918317 

Epoch 60/60, Batch 13 loss is 0.0014473636401817203 

Epoch 60/60, Batch 14 loss is 0.00169

100%|██████████| 61/61 [12:56<00:00, 12.74s/it]

Epoch 60/60, Batch 116 loss is 0.0014194046379998326 

Epoch 60/60, Batch 117 loss is 0.002434339141473174 

Loss is 0.0013934381018092825


Finish training.
